# Country by commodity exports

In [1]:
from gssutils import *

scraper = Scraper('https://www.ons.gov.uk/economy/nationalaccounts/balanceofpayments/datasets/' + \
                 'uktradecountrybycommodityexports')
scraper

## Trade in goods: country-by-commodity exports

Monthly export country-by-commodity data on the UK's trade in goods, including trade by all countries and selected commodities, non-seasonally adjusted.

### Distributions

1. Trade in goods: country-by-commodity exports ([MS Excel Spreadsheet](https://www.ons.gov.uk/file?uri=/economy/nationalaccounts/balanceofpayments/datasets/uktradecountrybycommodityexports/current/countrybycommodityexportsfinal.xlsx))


In [2]:
table = scraper.distribution().as_pandas(dtype={
    'COMMODITY': 'category',
    'COUNTRY': 'category',
    'DIRECTION': 'category'
}, na_values=[''], keep_default_na=False)
table

,COMMODITY,COUNTRY,DIRECTION,1998JAN,1998FEB,1998MAR,1998APR,1998MAY,1998JUN,1998JUL,...,2018FEB,2018MAR,2018APR,2018MAY,2018JUN,2018JUL,2018AUG,2018SEP,2018OCT,2018NOV
0,0 Food & live animals,AD Andorra,EX Exports,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0 Food & live animals,AE United Arab Emirates,EX Exports,5,5,5,4,5,4,6,...,17,21,21,19,17,20,30,19,20,21
2,0 Food & live animals,AF Afghanistan,EX Exports,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0 Food & live animals,AG Antigua & Barbuda,EX Exports,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0 Food & live animals,AI Anguilla,EX Exports,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0 Food & live animals,AL Albania,EX Exports,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0 Food & live animals,AM Armenia,EX Exports,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0 Food & live animals,AO Angola,EX Exports,0,0,0,0,0,0,0,...,0,2,1,0,1,2,1,1,3,1
8,0 Food & live animals,AQ Antarctica,EX Exports,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0 Food & live animals,AR Argentina,EX Exports,0,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [3]:
table.drop(columns='DIRECTION', inplace=True)
table.rename(columns={
    'COMMODITY': 'CORD SITC',
    'COUNTRY': 'ONS Partner Geography'}, inplace=True)
table = pd.melt(table, id_vars=['CORD SITC','ONS Partner Geography'], var_name='Period', value_name='Value')
table['Period'] = table['Period'].astype('category')
table['Value'] = table['Value'].astype(int)

In [4]:
display(table['CORD SITC'].cat.categories)
display(table['ONS Partner Geography'].cat.categories)

Index(['0 Food & live animals', '00 Live animals',
       '01 Meat & meat preparations', '02 Dairy products & eggs',
       '03 Fish & shellfish', '04 Cereals', '05 Vegetables & fruit',
       '06 Sugar', '07 Coffee, tea, cocoa etc', '08 Animal feeding stuffs',
       ...
       '897C Jewellery', '89I Other manufactures (intermediate)',
       '89K Other manufactures (capital)',
       '89OC Other manufactures (consumer)', '8O Other manufactures',
       '8OC Other miscellaneous manufactures (consumer)',
       '8OI Other miscellaneous manufactures (intermediate)',
       '8OK Other miscellaneous manufactures (capital)', '9 Unspecified goods',
       'T Total'],
      dtype='object', length=125)

Index(['AD Andorra', 'AE United Arab Emirates', 'AF Afghanistan',
       'AG Antigua & Barbuda', 'AI Anguilla', 'AL Albania', 'AM Armenia',
       'AO Angola', 'AQ Antarctica', 'AR Argentina',
       ...
       'VN Vietnam', 'VU Vanuatu', 'W1 Whole world', 'WF Wallis & Futuna',
       'WS Samoa', 'XK Kosovo', 'YE Yemen', 'ZA South Africa', 'ZM Zambia',
       'ZW Zimbabwe'],
      dtype='object', length=237)

Fix up category strings

In [5]:
table['CORD SITC'].cat.categories = table['CORD SITC'].cat.categories.map(lambda x: x.split(' ')[0])
table['ONS Partner Geography'].cat.categories = table['ONS Partner Geography'].cat.categories.map(lambda x: x[:2])
display(table['CORD SITC'].cat.categories)
display(table['ONS Partner Geography'].cat.categories)

Index(['0', '00', '01', '02', '03', '04', '05', '06', '07', '08',
       ...
       '897C', '89I', '89K', '89OC', '8O', '8OC', '8OI', '8OK', '9', 'T'],
      dtype='object', length=125)

Index(['AD', 'AE', 'AF', 'AG', 'AI', 'AL', 'AM', 'AO', 'AQ', 'AR',
       ...
       'VN', 'VU', 'W1', 'WF', 'WS', 'XK', 'YE', 'ZA', 'ZM', 'ZW'],
      dtype='object', length=237)

In [6]:
import re
YEAR_RE = re.compile(r'[0-9]{4}')
YEAR_MONTH_RE = re.compile(r'([0-9]{4})(JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC)')
YEAR_QUARTER_RE = re.compile(r'([0-9]{4})\s+(Q[1-4])')

# from https://stackoverflow.com/questions/597476/how-to-concisely-cascade-through-multiple-regex-statements-in-python
class Re(object):
  def __init__(self):
    self.last_match = None
  def fullmatch(self,pattern,text):
    self.last_match = re.fullmatch(pattern,text)
    return self.last_match

def time2period(t):
    gre = Re()
    if gre.fullmatch(YEAR_RE, t):
        return f"year/{t}"
    elif gre.fullmatch(YEAR_MONTH_RE, t):
        year, month = gre.last_match.groups()
        month_num = {'JAN': '01', 'FEB': '02', 'MAR': '03', 'APR': '04', 'MAY': '05', 'JUN': '06',
                     'JUL': '07', 'AUG': '08', 'SEP': '09', 'OCT': '10', 'NOV': '11', 'DEC': '12'}.get(month)
        return f"month/{year}-{month_num}"
    elif gre.fullmatch(YEAR_QUARTER_RE, t):
        year, quarter = gre.last_match.groups()
        return f"quarter/{year}-{quarter}"
    else:
        print(f"no match for {t}")

table['Period'].cat.categories = table['Period'].cat.categories.map(time2period)

In [7]:
table['Seasonal Adjustment'] = pd.Series('NSA', index=table.index, dtype='category')
table['Measure Type'] = pd.Series('GBP Total', index=table.index, dtype='category')
table['Unit'] = pd.Series('gbp-million', index=table.index, dtype='category')
table['Flow'] = pd.Series('exports', index=table.index, dtype='category')

In [8]:
table.memory_usage()

Index                          80
CORD SITC                 7441995
ONS Partner Geography    14883886
Period                   14883998
Value                    59487000
Seasonal Adjustment       7435963
Measure Type              7435963
Unit                      7435963
Flow                      7435963
dtype: int64

In [9]:
table = table[['ONS Partner Geography', 'Period','Flow','CORD SITC', 'Seasonal Adjustment', 'Measure Type','Value','Unit' ]]

In [10]:
table.count()

ONS Partner Geography    7435875
Period                   7435875
Flow                     7435875
CORD SITC                7435875
Seasonal Adjustment      7435875
Measure Type             7435875
Value                    7435875
Unit                     7435875
dtype: int64

In [11]:
table.dtypes

ONS Partner Geography    category
Period                   category
Flow                     category
CORD SITC                category
Seasonal Adjustment      category
Measure Type             category
Value                       int64
Unit                     category
dtype: object